In [5]:
from card_games import TexasHoldem, Poker

In [6]:
p = TexasHoldem()

In [30]:
H = ['6♥', '2♥', '6♥', '6♥', 'A♥', '8♥', '2♥','3♥', '4♥', 'Q♥', 'K♥', '9♥', '5♥']
print(H)

['6♥', '2♥', '6♥', '6♥', 'A♥', '8♥', '2♥', '3♥', '4♥', 'Q♥', 'K♥', '9♥', '5♥']


In [8]:
p._sorted_rank(H)

['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']

In [9]:
p._sorted_suit(H)

['6♥', '2♥', '6♥', '6♥', 'A♥', '8♥', '2♥', '3♥', '4♥', 'Q♥', 'K♥', '9♥', '5♥']

In [10]:
p.sorted_hand(H)

['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']

In [11]:
p._sorted_suit(p._sorted_rank(H))

['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']

In [31]:
print(p.sorted_hand(H), '\n')
for (a,b),c in p.get_hand_categories().items():
    print(a,b,c(H), '\n')

['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥'] 

1 highest rank (['A♥'], ['K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']) 

2 pair (['6♥', '6♥'], ['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']) 

3 two pairs (['6♥', '6♥', '2♥', '2♥'], ['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '5♥', '4♥', '3♥']) 

4 three (['6♥', '6♥', '6♥'], ['A♥', 'K♥', 'Q♥', '9♥', '8♥', '5♥', '4♥', '3♥', '2♥', '2♥']) 

5 straight (['5♥', '4♥', '3♥', '2♥', 'A♥'], ['K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '2♥']) 

6 flush (['A♥', 'K♥', 'Q♥', '9♥', '8♥'], ['6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']) 

7 full house (['6♥', '6♥', '6♥', '2♥', '2♥'], ['A♥', 'K♥', 'Q♥', '9♥', '8♥', '5♥', '4♥', '3♥']) 

8 four ([], ['A♥', 'K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '5♥', '4♥', '3♥', '2♥', '2♥']) 

9 straight flush (['5♥', '4♥', '3♥', '2♥', 'A♥'], ['K♥', 'Q♥', '9♥', '8♥', '6♥', '6♥', '6♥', '2♥']) 

10 royal straight flush ([], ['A♥', 'K♥', 'Q♥', '9♥', 

In [13]:
p.evaluate_hand(H)

((9, 'straight flush'),
 (5, 4, 3, 2, 14),
 (13, 12, 9, 8, 6, 6, 6, 2),
 (3, 3, 3, 3, 3),
 (3, 3, 3, 3, 3, 3, 3, 3))

In [24]:
for i in range(10000):
    p = Poker(num_players=8)
    p.shuffle()
    r = p.evaluate_players()
    if len(p.winners().items())>1:
        break
r

{0: ((1, 'highest rank'), (14,), (7, 4, 3, 2), (1,), (1, 3, 1, 4)),
 1: ((1, 'highest rank'), (11,), (10, 8, 4, 3), (4,), (1, 4, 2, 2)),
 2: ((1, 'highest rank'), (14,), (13, 8, 6, 2), (2,), (3, 2, 2, 2)),
 3: ((1, 'highest rank'), (8,), (7, 5, 3, 2), (1,), (3, 1, 4, 3)),
 4: ((1, 'highest rank'), (9,), (8, 6, 5, 4), (3,), (3, 3, 3, 1)),
 5: ((5, 'straight'), (13, 12, 11, 10, 9), (), (4, 1, 1, 4, 4), ()),
 6: ((5, 'straight'), (13, 12, 11, 10, 9), (), (1, 2, 3, 3, 1), ()),
 7: ((2, 'pair'), (5, 5), (11, 7, 4), (4, 2), (2, 4, 4))}

In [25]:
p.winners()

{5: ((5, 'straight'), (13, 12, 11, 10, 9), (), (4, 1, 1, 4, 4), ()),
 6: ((5, 'straight'), (13, 12, 11, 10, 9), (), (1, 2, 3, 3, 1), ())}

In [26]:
p.winner()

(6, ((5, 'straight'), (13, 12, 11, 10, 9), (), (1, 2, 3, 3, 1), ()))

In [27]:
p.players

[['2♠', '7♣', 'A♣', '4♥', '3♣'],
 ['8♠', 'T♣', 'J♠', '3♦', '4♦'],
 ['2♦', '6♦', 'K♥', '8♦', 'A♦'],
 ['8♣', '3♠', '7♥', '2♥', '5♣'],
 ['5♥', '8♥', '6♥', '9♥', '4♣'],
 ['J♣', '9♠', 'Q♣', 'K♠', 'T♠'],
 ['K♣', 'J♥', 'T♥', '9♣', 'Q♦'],
 ['7♠', '4♠', '5♠', '5♦', 'J♦']]

In [28]:
p.table

[]